In [1]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
import numpy as np
import scipy.stats as si
import sympy as sy
from sympy.stats import Normal, cdf
from sympy import init_printing
from pandas import DataFrame
from datetime import date
import pymongo
import pandas as pd

In [2]:
# Underlying price (per share): S; 
# Strike price of the option (per share): K;
# Time to maturity (years): T;(To be given in years)
# Continuously compounding risk-free interest rate: r;(Annual in % to be given)
# Volatility: sigma;(in % to be given)

def bs(S,K,r,T,sigma,x):
    d1 = (np.log(S / K) + (r + 0.5 * (sigma ** 2)) * T) / (sigma * np.sqrt(T))
    d2 = d1-sigma*np.sqrt(T)
    prob_density = 1 / np.sqrt(2 * np.pi) * np.exp(-(d1 ** 2) * 0.5)
    result= pd.DataFrame()
    result['Premium'] = (S *x* si.norm.cdf(d1*x, 0.0, 1.0) - K *x* np.exp(-r * T) * si.norm.cdf(d2*x, 0.0, 1.0))
    result['Delta'] = si.norm.cdf(d1*x, 0.0, 1.0)*x
    #result['Theta'] = (-sigma * S * prob_density) / (2 * np.sqrt(T)) - r * K*x * np.exp(-r * T) * si.norm.cdf(d2*x, 0.0, 1.0) 
    result['Theta'] = 0.01*(-(S*prob_density*sigma)/(2*np.sqrt(T)) - r*K*x*np.exp(-r*T)*si.norm.cdf(-d2*x,0.0,1.0))
    result['Gamma'] = prob_density / (S * sigma * np.sqrt(T))
    result['Vega'] = 0.01*S * prob_density * np.sqrt(T)
    result['Rho'] = 0.01*T * K*x * np.exp(-r * T) * si.norm.cdf(d2*x, 0.0, 1.0)
    return result

In [6]:
#s=Stock;o=Option data
#To fetch Data
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["OHLC_MINUTE_1"]
mycol_s = mydb['StockData']
mycol_o = mydb['OptionData']
df_s = pd.DataFrame(list(mycol_s.find())) #Dataframe Created of OHLC Data
df_o = pd.DataFrame(list(mycol_o.find())) #Dataframe Created of OHLC Data
df_s.set_index("LastTradeTime", inplace = True) 
df_o.set_index("LastTradeTime", inplace = True) 
df=pd.DataFrame()
df['StockPrice']=df_s['Open']
df['OptionPremium']=df_o['Open']


In [7]:
import datetime
#Converting to Epoch Time
date_time_str = '20'+ year + month + date + '15:30:00'
ExpiryTime = datetime.datetime.strptime(date_time_str, '%Y%B%d%H:%M:%S').timestamp()
df['Expiry Days'] = (ExpiryTime-df.index)/(86400)
df=df.dropna()
#For Call Options Input is 1; For Put Option: Input is -1
if(type=='CE'):
    df['type']=1
else:
    df['type']=-1

In [9]:
Rate=0.048
VIX=40
#Days to be converted in form of years, VIX and Rate to be in %
callResults=bs(df['StockPrice'],StrikePrice,Rate,df['Expiry Days']/365 ,VIX/100, 1)
callResults.head(5)

,Premium,Delta,Theta,Gamma,Vega,Rho
LastTradeTime,,,,,,
1588909560,170.091641,0.273725,-108.080635,0.000321,8.641097,0.901275
1588909620,190.739611,0.297358,-112.297195,0.000332,9.020981,0.981650
1588909680,181.832604,0.287314,-110.572608,0.000328,8.863916,0.947314
1588909740,173.506646,0.277741,-108.847796,0.000323,8.707262,0.914636
1588909800,167.481701,0.270699,-107.529455,0.000319,8.587557,0.890611
